# Assignment 4 #

We are given three columns of integers with a row for each node. The first two columns contain x and y coordinates of the node positions in a plane. The third column contains node costs. The goal is to select exactly 50% of the nodes (if the number of nodes is odd we round the number of nodes to be selected up) and form a Hamiltonian cycle (closed path) through this set of nodes such that the sum of the total length of the path plus the total cost of the selected nodes is minimized. The distances between nodes are calculated as Euclidean distances rounded mathematically to integer values. The distance matrix should be calculated just after reading an instance and then only the distance matrix (no nodes coordinates) should be accessed by optimization methods to allow instances defined only by distance matrices. 

## Read the data ##

In [1]:
import pandas as pd
import numpy as np
from numpy.typing import ArrayLike, NDArray
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from math import sqrt
from typing import List, Optional, Tuple, Union 

In [2]:
# read data into dataframes
instances = {
    "A": pd.read_csv("data/TSPA.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "B": pd.read_csv("data/TSPB.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "C": pd.read_csv("data/TSPC.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "D": pd.read_csv("data/TSPD.csv", sep=';', header=None, names=["x", "y", "cost"]),
}

In [3]:
def calculate_distance_matrix(df: pd.DataFrame) -> NDArray[np.int32]:
    """
    Calculate the distance matrix from the dataframe.
    The dataframe contains 'x' and 'y' columns for the coordinates.
    The distances are Euclidean, rounded to the nearest integer + the cost of the destination node.
    """
    coordinates = df[['x', 'y']].to_numpy()
    dist_matrix = np.zeros(shape=(len(df), len(df)))
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            dist_matrix[i, j] = round(sqrt((coordinates[i, 0] - coordinates[j, 0])**2 + (coordinates[i, 1] - coordinates[j, 1])**2))
    return dist_matrix

In [12]:
def calculate_nearest_vertices(dist_matrix: np.ndarray, costs: np.ndarray, num_candidates: int = 10) -> dict:
    num_vertices = dist_matrix.shape[0]
    nearest_vertices = []

    for i in range(num_vertices):
        cost_distance_sum = dist_matrix[i, :] + costs
        sorted_vertices = np.argsort(cost_distance_sum)
        nearest_vertices.append([vertex for vertex in sorted_vertices if vertex != i][:num_candidates])

    return nearest_vertices

In [13]:
distances_matrices = {
    "A": calculate_distance_matrix(instances["A"]),
    "B": calculate_distance_matrix(instances["B"]),
    "C": calculate_distance_matrix(instances["C"]),
    "D": calculate_distance_matrix(instances["D"])
}

costs = {
    "A": instances["A"]["cost"].to_numpy(),
    "B": instances["B"]["cost"].to_numpy(),
    "C": instances["C"]["cost"].to_numpy(),
    "D": instances["D"]["cost"].to_numpy()
}

nearest_vertices = {
    "A": calculate_nearest_vertices(distances_matrices["A"], costs["A"]),
    "B": calculate_nearest_vertices(distances_matrices["B"], costs["B"]),
    "C": calculate_nearest_vertices(distances_matrices["C"], costs["C"]),
    "D": calculate_nearest_vertices(distances_matrices["D"], costs["D"])
}

In [5]:
def visualize_selected_route(
    selected_nodes_indices: ArrayLike, 
    dataframe: pd.DataFrame,
    title: str) -> None:
    """
    Visualize the selected route returned by the algorithm, including the cost of each node represented by a colormap.

    Parameters:
    selected_nodes_indices (list): Indices of the selected nodes in the route.
    dataframe (DataFrame): DataFrame containing 'x', 'y', and 'cost' columns for each node.
    """
    x = dataframe["x"].to_numpy()
    y = dataframe["y"].to_numpy()
    costs = dataframe["cost"].to_numpy()

    cmap = plt.cm.get_cmap('viridis')
    norm = plt.Normalize(vmin=min(costs), vmax=max(costs))

    plt.figure(figsize=(15, 10))
    scatter = plt.scatter(x, y, c=costs, cmap=cmap, norm=norm, s=100)
    plt.colorbar(scatter, label='Node Cost')

    for i, node in enumerate(selected_nodes_indices):
        start_node = selected_nodes_indices[i]
        end_node = selected_nodes_indices[(i + 1) % len(selected_nodes_indices)]
        plt.plot([x[start_node], x[end_node]], [y[start_node], y[end_node]], 'k-', lw=1)

    plt.title(title, fontsize=18)
    plt.xlabel('X Coordinate', fontsize=14)
    plt.ylabel('Y Coordinate', fontsize=14)
    plt.grid(True)
    plt.show()

In [6]:
def objective_function(solution: list[int], dist_matrix: list[list[int]], costs: list[int]) -> int:
    total_score = 0
    n = len(solution)
    for x in range(n):
        total_score += dist_matrix[solution[x - 1]][solution[x]]
        total_score += costs[solution[x]]
    return total_score

In [7]:
# A function that generates a random solution
def generate_random_solution(n: int) -> list[int]:
    """
    Generate a random solution for a given number of nodes.

    :param n: The number of nodes.
    :return: A list of nodes representing the solution.
    """
    return random.sample(range(0, n * 2), n)

In [40]:
def two_edges_exchange(current_solution: list[int], 
                       nearest_vertices: list[list[int]]) -> list[list[int]]:
    """
    Generate new solutions by exchanging two edges in the current solution.

    :param current_solution: List of nodes in the current solution.
    :param current_score: The score of the current solution.
    :param distance_matrix: 2D list representing the distances between nodes.
    :return: A list of tuples where each tuple contains a new solution and its score.
    """
    n = len(current_solution)
    new_solutions = []

    for i in range(n):
        node1 = current_solution[i]
        node1_index = i
        node2_candidates = nearest_vertices[node1]
        
        for node2 in node2_candidates:
            if node2 in current_solution:
                node2_index = current_solution.index(node2)
                
                if (node2_index - node1_index) % n > 1:
                    new_solution1 = (current_solution[:node1_index + 1] 
                                    + current_solution[node1_index + 1:node2_index + 1][::-1] 
                                    + current_solution[node2_index + 1:])
                    
                    new_solution2 = (current_solution[:node1_index + 1] 
                                    + current_solution[node2_index:] 
                                    + current_solution[node1_index + 1:node2_index][::-1])
                    
                    new_solutions.append(new_solution1)
                    new_solutions.append(new_solution2)
    
    return new_solutions

In [41]:
sol = [2,7,4,6,1,5,3,8,0]
nearest_vertics = [
    [5],[],[],[],[],[],[],[],[]
]
two_edges_exchange(sol, nearest_vertics)

[[2, 7, 4, 6, 1, 5, 3, 8, 0, 3, 8, 0], [2, 7, 4, 6, 1, 5, 3, 8, 0, 5, 3, 8, 0]]

In [527]:
def two_edges_exchange_advanced(current_solution: list[int], 
                                current_distance: float, 
                                distance_matrix: list[list[int]],
                                start_index: int = 0,
                                direction: str = "right"):
    """
    Generate a new solution by exchanging two edges in the current solution,
    starting from a given index and moving in the specified direction.

    :param current_solution: List of nodes in the current solution.
    :param current_distance: The score of the current solution.
    :param distance_matrix: 2D list representing the distances between nodes.
    :param start_index: The index from which to start searching for a better solution.
    :param direction: The direction in which to search for a better solution ("right" or "left").
    :return: A tuple containing the new solution and its score if it's better,
             otherwise the original solution and score.
    """
    n = len(current_solution)

    # Define the order of iteration based on the direction
    if direction == "right":
        range_i = range(n - 2)
        range_j = lambda i: range(i + 2, n)
    else:  # direction == "left"
        range_i = range(n - 3, -1, -1)
        range_j = lambda i: range(n - 1, i + 1, -1)

    # Convert the linear start index to a pair of indices (i, j)
    count = 0
    for i in range_i:
        for j in range_j(i):
            if count >= start_index:
                # Perform the two-edges exchange from this point
                new_solution = (current_solution[:i + 1] 
                                + current_solution[i + 1:j + 1][::-1] 
                                + current_solution[j + 1:])

                score_delta = (
                    -distance_matrix[current_solution[i]][current_solution[i + 1]]
                    -distance_matrix[current_solution[j]][current_solution[(j + 1) % n]]
                    +distance_matrix[current_solution[i]][current_solution[j]]
                    +distance_matrix[current_solution[i + 1]][current_solution[(j + 1) % n]]
                )
                new_score = current_distance + score_delta

                if new_score < current_distance:
                    return new_solution, new_score

            count += 1  # Increment the counter after checking the condition

    return None


In [528]:
def two_nodes_exchange(current_solution, score, distance_matrix, start_index=0, direction='right'):
    n = len(current_solution)
    total_moves = n * (n - 1) // 2  # Total number of possible swaps

    index_pairs = [(x, y) for x in range(n) for y in range(x+1, n)]
    # Adjust the indices list based on the direction
    if direction == 'left':
        index_pairs = index_pairs[::-1]
        start_index = total_moves - start_index - 1 

    for count, (i, j) in enumerate(index_pairs[start_index:], start=start_index):
        temp = current_solution[:]
        temp_score = score

        if i == 0 and j == n - 1:  # special case: first and last nodes
            score_delta = (
                -distance_matrix[current_solution[j]][current_solution[0]]
                -distance_matrix[current_solution[j-1]][current_solution[j]]
                -distance_matrix[current_solution[0]][current_solution[1]]
                +distance_matrix[current_solution[j]][current_solution[1]]
                +distance_matrix[current_solution[j-1]][current_solution[0]]
                +distance_matrix[current_solution[0]][current_solution[j]]
            )
        elif j == i + 1:  # adjacent nodes case
            score_delta = (
                -distance_matrix[current_solution[i - 1]][current_solution[i]]
                -distance_matrix[current_solution[j]][current_solution[(j + 1) % n]]
                +distance_matrix[current_solution[i - 1]][current_solution[j]]
                +distance_matrix[current_solution[i]][current_solution[(j + 1) % n]]
            )
        else:  # non-adjacent nodes case
            score_delta = (
                -distance_matrix[current_solution[i - 1]][current_solution[i]]
                -distance_matrix[current_solution[j - 1]][current_solution[j]]
                +distance_matrix[current_solution[i - 1]][current_solution[j]]
                +distance_matrix[current_solution[j - 1]][current_solution[i]]
                -distance_matrix[current_solution[i]][current_solution[(i + 1) % n]]
                -distance_matrix[current_solution[j]][current_solution[(j + 1) % n]]
                +distance_matrix[current_solution[i]][current_solution[(j + 1) % n]]
                +distance_matrix[current_solution[j]][current_solution[(i + 1) % n]]
            )

        temp[i], temp[j] = temp[j], temp[i]
        temp_score += score_delta
        # If the new score is better, return the new solution immediately
        if temp_score < score:
            return temp, temp_score
    # If no improvement is found, return None
    return None, None

In [531]:
def inter_route_exchange_simple(selected, unselected, score, distance_matrix, costs):
    new_solutions = []

    # Assume node indices are 0-based for distance_matrix
    for selected_node in selected:
        for new_node in unselected:
            new_solution = selected.copy()
            replaced_node_index = selected.index(selected_node)
            
            # Assuming 'selected' and 'unselected' contain 0-based indices already
            new_solution[replaced_node_index] = new_node

            prev_node_index = (replaced_node_index - 1) % len(selected)
            next_node_index = (replaced_node_index + 1) % len(selected)

            # Calculate score_delta considering 0-based indices for distance_matrix
            score_delta = (
                -distance_matrix[selected[prev_node_index]][selected_node]
                -distance_matrix[selected_node][selected[next_node_index]]
                +distance_matrix[selected[prev_node_index]][new_node]
                +distance_matrix[new_node][selected[next_node_index]]
                -costs[selected_node]
                +costs[new_node]
            )
            new_score = score + score_delta

            new_solutions.append((new_solution, new_score))

    return new_solutions


In [532]:
def inter_route_exchange(current_solution, unselected_nodes, distance_matrix, costs, start_index=0, direction="right"):
    n_selected = len(current_solution)
    n_unselected = len(unselected_nodes)
    current_score = objective_function(current_solution, distance_matrix, costs)
    # Create all possible combinations of selected and unselected nodes
    all_combinations = [(i, j) for i in range(n_selected) for j in range(n_unselected)]
    if direction == "left":
        all_combinations = all_combinations[::-1]
    for i, j in all_combinations[start_index:]:
        selected_node = current_solution[i]
        new_node = unselected_nodes[j]
        new_solution = current_solution.copy()
        new_solution[i] = new_node
        prev_node_index = (i - 1) % n_selected
        next_node_index = (i + 1) % n_selected
        score_delta = (
            -distance_matrix[current_solution[prev_node_index]][selected_node]
            -distance_matrix[selected_node][current_solution[next_node_index]]
            +distance_matrix[current_solution[prev_node_index]][new_node]
            +distance_matrix[new_node][current_solution[next_node_index]]
            -costs[selected_node]
            +costs[new_node]
        )
        new_score = current_score + score_delta
        if new_score < current_score:
            # remove from unselected nodes the new inserted ine
            unselected_nodes.remove(new_node)
            # add to unselected the node that has been dropped
            unselected_nodes.append(selected_node)
            return new_solution, new_score
    # If no better solution is found, return None
    return None, None

# Repository for Local Search solution finder #

In [7]:
from abc import ABC, abstractmethod
from typing import Union, Optional


class LocalSearch(ABC):
    """
    Abstract class for local search algorithms.
    """
    def __init__(self, 
                 initial_solution: list[int], 
                 distance_matrix: list[list[int]], 
                 costs: list[int]) -> None:
        """
        :param initial_solution: The initial solution.
        :param initial_score: The score of the initial solution.
        :param distance_matrix: 2D list representing the distances between nodes.
        """
        self.initial_solution = initial_solution
        self.initial_score = objective_function(initial_solution, distance_matrix, costs)
        self.distance_matrix = distance_matrix
        self.costs = costs
        
        self.all_nodes = list(range(100*2))
        self.unselected_nodes = [node for node in self.all_nodes if node not in self.initial_solution]
        
        self.current_solution: list[int] = initial_solution
        self.current_score = self.initial_score
        
        self.moves = {
            "intra-two-edges-exchange": self.two_edges_exchange,
            "intra-two-nodes-exchange": self.two_nodes_exchange,
            "inter-route-exchange": self.inter_route_exchange
        }
        
        self.moves_utils = {
            "intra-two-edges-exchange": self.get_utils_intra_two_edges_exchange,
            "intra-two-nodes-exchange": self.get_utils_intra_two_nodes_exchange,
            "inter-route-exchange": self.get_utils_inter_route_exchange
        }
        
    
    @abstractmethod
    def candidates_moves
        
        
    @abstractmethod
    def get_utils_intra_two_edges_exchange(self):
        raise NotImplementedError
    
    @abstractmethod
    def get_utils_intra_two_nodes_exchange(self):
        raise NotImplementedError
    
    @abstractmethod
    def get_utils_inter_route_exchange(self):
        raise NotImplementedError

    # two method for intra moves
    @abstractmethod
    def two_edges_exchange(self, start_index: int = 0, direction: str = "right") -> Union[tuple[list[int], int], None]:
        """This method should return a new solution and its score if a better solution is found, otherwise None.
        For a greedy algorithm, the first better solution found should be returned. For a steepest local search,
        the best solution among all possible solutions should be returned, if there is no better solution, just
        return None. For a steepest local search, both neighborhood moves will return one solution, then we will
        take the better one among these two solutions.

        Parameters
        ----------
        current_solution : list[int]
        current_distance : int
        distance_matrix : list[list[int]]

        Returns
        -------
        tuple[list[int], int] | None
            A single better solution founded by the algorithm, or None if no better solution is found.
        """
        raise NotImplementedError
    
    @abstractmethod
    def two_nodes_exchange(self, start_index=0, direction='right') -> Union[tuple[list[int], int] , None]:
        raise NotImplementedError
    
    # one method for inter move
    @abstractmethod
    def inter_route_exchange(self, start_index=0, direction="right") -> Union[tuple[list[int], int] , None]:
        raise NotImplementedError
    
    @abstractmethod
    def run(self, *args, **kwargs) -> None:
        """
        Run the algorithm.
        """
        pass

In [8]:
class CandidateLocalSearch(LocalSearch):
    def __init__(self, 
                 initial_solution: list[int], 
                 distance_matrix: list[list[int]], 
                 costs: list[int]) -> None:
        super().__init__(initial_solution, distance_matrix, costs)
        self.nearest_vertices = self.candidate_nodes

    def candidate_nodes(self, 
                        num_candidates):
        num_sol = len(self.distance_matrix[0])
        distance_and_cost=self.distance_matrix + self.costs.reshape(-1,1)
        self.candidates = [[] for x in range(num_sol)]
        for x in range( num_sol ):
           temp = sorted(range(num_sol), key=lambda k: distance_and_cost[k])[:num_candidates+1]
           temp.remove(x)
           self.candidates[x] = temp 


    def two_edges_exchange(self,
                        start_index: int = 0,
                        direction: str = "right"):
        n = len(self.current_solution)


        for n1 in range(n-1):
            for n2 in range(self.candidates[n1]):
                part1=self.current_solution[:n1+1]
                
                

        return None, None


        